## import 

In [22]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
import time
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import clear_output

In [24]:
expiry_day = 'BANKNIFTY23803'
lower_price = 45500
upper_price = 43500


inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march = (
    inst[(inst.name == "BANKNIFTY") & (inst.tradingsymbol.str.contains(expiry_day))]
    .query("(strike >= @lower_price )&( strike <= @upper_price)")
    .assign(strike_type=lambda x: x["strike"].astype(int).astype(str) + x["instrument_type"])
)

In [26]:
expiry_march.shape

(0, 13)

In [42]:
def get_data_minute(expiry_march, start_dt, end_dt, time_frame, bn_token):
    df_combined = pd.DataFrame()
    strike_symbol_dict = dict(
        zip(expiry_march.instrument_token, expiry_march["strike_type"])
    )
    strike_symbol_dict[bn_token] = "BANKNIFTY"
    inst_expiry = expiry_march["instrument_token"].unique().tolist()

    for i in inst_expiry:
        df_combined = df_combined.append(
            pd.DataFrame(
                kite.historical_data(
                    i,
                    from_date=start_dt,
                    to_date=end_dt,
                    interval=time_frame,
                    continuous=False,
                    oi=True,
                )
            ).assign(strike_type=strike_symbol_dict[i])
        )
    df_bn_hist_data = (
        pd.DataFrame(
            kite.historical_data(
                bn_token,
                from_date=start_dt,
                to_date=end_dt,
                interval=time_frame,
                continuous=False,
                oi=True,
            )
        )
        .assign(strike_type="BANKNIFTY")
        .drop("oi", axis=1)
        .assign(oi_change=lambda x: x["close"])
    )

    # df_bn_hist_data['oi_change'] = df_bn_hist_data.groupby(['strike_type']).oi.diff()

    df_combined = df_combined.assign(
        oi_change=lambda x: x.groupby("strike_type")["oi"].diff()
    ).assign(oi_change=lambda x: x.groupby("strike_type")["oi_change"].cumsum())

    df_combined_1 = (
        df_combined.groupby("date")
        .agg({"oi_change": lambda x: sorted(x, reverse=True)[0:5]})
        .reset_index()
        .explode("oi_change")
    )

    # top 3 CE and PE
    df_combined_ce = (
        df_combined.query('strike_type.str[-2:] == "CE"')
        .groupby("date")
        .agg({"oi_change": lambda x: sorted(x, reverse=True)[0:3]})
        .reset_index()
        .explode("oi_change")
        .merge(
            df_combined[["date", "oi_change", "strike_type"]],
            how="left",
            on=["date", "oi_change"],
        )
        .dropna()
    )

    df_combined_pe = (
        df_combined.query('strike_type.str[-2:] == "PE"')
        .groupby("date")
        .agg({"oi_change": lambda x: sorted(x, reverse=True)[0:3]})
        .reset_index()
        .explode("oi_change")
        .merge(
            df_combined[["date", "oi_change", "strike_type"]],
            how="left",
            on=["date", "oi_change"],
        )
        .dropna()
    )

    df_combined_ce_pe = df_combined_ce.append(df_combined_pe)

    req_cols = df_combined_1.columns
    df_combined_1 = df_combined_1.append(df_bn_hist_data[req_cols]).rename(
        columns={"oi_change": "oi"}
    )
    df_combined_ce_pe = df_combined_ce_pe.append(df_bn_hist_data[req_cols]).rename(
        columns={"oi_change": "oi"}
    )
    # df_combined = df_combined.drop('oi',axis=1).append(df_bn_hist_data[req_cols]).rename(columns = {'oi_change':'oi'})
    return df_combined, df_combined_1, df_bn_hist_data, df_combined_ce_pe


In [43]:
df_combined, df_combined_1, df_bn_hist_data, df_combined_ce_pe = get_data_minute(
    expiry_march, start_dt, end_dt, time_frame, bn_token
)


In [45]:
df_combined

""


In [18]:
# d1 = df_combined.set_index(['date', 'strike_type']).oi.unstack(['strike_type'])
# d1 = d1.pct_change().stack([0]).reset_index()
# df_combined = df_combined.merge(d1, on=['date', 'strike_type'], how='left')
# df_combined.rename(columns={0: 'pct'}, inplace=True)
# df_combined['strike_price'] = df_combined.strike_type.str[:5].astype(int)
# df_combined['side'] = df_combined.strike_type.str[5:]
# df_combined['date'] = df_combined['date'].dt.tz_localize(None)
# pd.pivot_table(df_combined,index = 'strike_price', columns=['date','side'], values='pct').to_excel('oi_pivot.xlsx')

## oi plot chg

In [41]:
start_dt = '2023-08-02'
end_dt = '2023-08-03'
time_frame = '5minute'
bn_token  = 260105
xx
while(True):
    df_combined,df_combined_1,df_bn_hist_data = get_data_minute(expiry_march, start_dt,end_dt,time_frame,bn_token)
    print('run')
    df_temp = df_bn_hist_data.copy()
    for t in df_combined_1['date'].unique()[-1:]:
        # selecting top 5 sp
        st_pr = df_combined_1.query('date == @t')['strike_type'].unique()

        # getting all prev OI of top 5 
        df_current_time_all =(
             df_combined[['date','oi','strike_type']].query('(date <= @t)')
             .query('strike_type in @st_pr')
             .sort_values(['date','oi'], ascending=False,)
        )
        sp = df_current_time_all.strike_type.unique()
        df_bn_hist_data = df_temp.query('(date <= @t)').copy()

        # df_current_time_all = df_current_time_all[~((df_current_time_all.strike_type == 'BANKNIFTY') & (df_current_time_all.oi == 0))]
        
        
        fig = go.Figure()
        data = []
        for i in range(0,5):
            data.append(go.Scatter(x=df_current_time_all.query("strike_type == @sp[@i]")['date'],
                                        y=df_current_time_all.query("strike_type == @sp[@i]")['oi'],
                                        name = sp[i],
                                        yaxis='y1'
                                        ))

        # data.append(go.Scatter(x=df_current_time_all.query("strike_type == @sp[5]")['date'],
        #                             y=df_current_time_all.query("strike_type == @sp[5]")['oi'],
        #                             name = sp[5],
        #                             yaxis='y2'
        #                             ))

        data.append(go.Candlestick(x=df_bn_hist_data.date,
                open=df_bn_hist_data.open,
                high=df_bn_hist_data.high,
                low=df_bn_hist_data.low,
                close=df_bn_hist_data.close,
                name = "BN",
                    yaxis='y2'    
                            ))
        layout = go.Layout(title='OI vs Bank',
                        yaxis=dict(title='Oi',),
                        yaxis2=dict(title='bank',
                                    overlaying='y',
                                    side='right',
                                    tickformat = '0.0f', 
                                    #range =[39000,44000]
                                      )
                        )
                                
                                    
        
        fig = go.Figure(data=data, layout=layout)
        fig.update_layout(height=600, width=1200)
        fig.show()
        clear_output(wait=True)   
        time.sleep(60)
        break
    # break


NameError: name 'xx' is not defined

In [16]:
df_combined_1['date'].unique()[-2:]

<DatetimeArray>
['2023-05-24 12:50:00+05:30', '2023-05-24 09:15:00+05:30']
Length: 2, dtype: datetime64[ns, tzoffset(None, 19800)]

In [18]:
start_dt = '2023-05-24'
end_dt = '2023-05-24'
time_frame = '5minute'
bn_token  = 260105

while(True):
    df_combined,df_combined_1,df_bn_hist_data,df_combined_ce_pe = get_data_minute(expiry_march, start_dt,end_dt,time_frame,bn_token)
    print('run')
    df_combined_1 = df_combined_ce_pe.copy()
    df_temp = df_bn_hist_data.copy()
    for t in df_combined_1['date'].unique()[-2:]:
        # selecting top 5 sp
        df_current_time = df_combined_1.query('date == @t')
        st_pr = df_current_time['strike_type'].unique()

        # getting all prev OI of top 5 
        df_current_time_all = df_combined[['date','oi','strike_type']].query('(date <= @t)')
        df_bn_hist_data = df_temp.copy()
        df_bn_hist_data = df_bn_hist_data.query('(date <= @t)')
        df_current_time_all = df_current_time_all[df_current_time_all['strike_type'].isin(st_pr)]
        # df_current_time_all = df_current_time_all[~((df_current_time_all.strike_type == 'BANKNIFTY') & (df_current_time_all.oi == 0))]
        df_current_time_all.sort_values(['date','oi'], ascending=False, inplace = True)
        sp = df_current_time_all.strike_type.unique()
        
        fig = go.Figure()
        fig_1 = go.Figure()
        data = []
        for i in range(0,6):
            data.append(go.Scatter(x=df_current_time_all.query("strike_type == @sp[@i]")['date'],
                                        y=df_current_time_all.query("strike_type == @sp[@i]")['oi'],
                                        name = sp[i],
                                        yaxis='y1'
                                        ))

        # data.append(go.Scatter(x=df_current_time_all.query("strike_type == @sp[5]")['date'],
        #                             y=df_current_time_all.query("strike_type == @sp[5]")['oi'],
        #                             name = sp[5],
        #                             yaxis='y2'
        #                             ))

        # data.append(go.Candlestick(x=df_bn_hist_data.date,
        #         open=df_bn_hist_data.open,
        #         high=df_bn_hist_data.high,
        #         low=df_bn_hist_data.low,
        #         close=df_bn_hist_data.close,
        #         name = "BN",
        #             yaxis='y2'    
        #                     ))
        layout = go.Layout(title='OI vs Bank',
                        yaxis=dict(title='Oi',),
                        # yaxis2=dict(title='bank',
                        #             overlaying='y',
                        #             side='right',
                        #             tickformat = '0.0f', 
                        #             #range =[39000,44000]
                        #               )
                        )
                                
                                    
        
        fig = go.Figure(data=data, layout=layout)
        fig.update_layout(height=600, width=1200)
        fig.show()
        clear_output(wait=True)   
        time.sleep(60)
        break
    # break


run


IndexError: index 1 is out of bounds for axis 0 with size 1